In [ ]:
!nvidia-smi

Wed May 25 07:52:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Clone and configure Darknet 
---

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15420 (delta 1), reused 6 (delta 1), pack-reused 15413
Receiving objects: 100% (15420/15420), 14.08 MiB | 24.14 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

## Configure YOLOv3 training parameters
---

In [ ]:
!cp /content/darknet/cfg/yolov3.cfg /content/darknet/cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' /content/darknet/cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' /content/darknet/cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 8000/' /content/darknet/cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=4@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=27@' /content/darknet/cfg/yolov3_training.cfg

In [ ]:
!echo -e 'holothurian\nechinus\nscallop\nstarfish' > data/obj.names

In [ ]:
!echo -e 'classes=4\ntrain=data/train.txt\nvalid=data/test.txt\nnames=data/obj.names\nbackup=/mydrive/yolov3_normal' > data/obj.data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive

## Get train dataset
---

In [ ]:
#change file name depending on the version of DUO being used
!unzip /mydrive/yolo_train.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/4730.jpg       
  inflating: data/obj/4730.txt       
  inflating: data/obj/4731.jpg       
  inflating: data/obj/4731.txt       
  inflating: data/obj/4732.jpg       
  inflating: data/obj/4732.txt       
  inflating: data/obj/4733.jpg       
  inflating: data/obj/4733.txt       
  inflating: data/obj/4734.jpg       
  inflating: data/obj/4734.txt       
  inflating: data/obj/4735.jpg       
  inflating: data/obj/4735.txt       
  inflating: data/obj/4736.jpg       
  inflating: data/obj/4736.txt       
  inflating: data/obj/4737.jpg       
  inflating: data/obj/4737.txt       
  inflating: data/obj/4738.jpg       
  inflating: data/obj/4738.txt       
  inflating: data/obj/4739.jpg       
  inflating: data/obj/4739.txt       
  inflating: data/obj/474.jpg        
  inflating: data/obj/474.txt        
  inflating: data/obj/4740.jpg       
  inflating: data/obj/4740.txt       
  inflating: data/obj/4741.jpg       

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

## Get YOLOv3 intial weights trained on ImageNet
---

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-05-24 13:18:27--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  18.4MB/s    in 9.5s    

2022-05-24 13:18:37 (16.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



## Training
---

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# To continue training from last weights if it haults in between
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3_normal/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 66856, rewritten_bbox = 0.180986 % 
]2;7926/8000: loss=1.7 hours left=0.3
 7926: 1.655321, 1.652633 avg loss, 0.001000 rate, 10.157598 seconds, 507264 images, 0.327486 hours left
Loaded: 0.000092 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000003, iou_loss = 0.000000, total_loss = 0.000003 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.861440), count: 4, class_loss = 0.394007, iou_loss = 0.049820, total_loss = 0.443827 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.789570), count: 33, class_loss = 3.693060, iou_loss = 1.212462, total_loss = 4.905521 
 total_bbox = 66893, rewritten_bbox = 0.180886 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000048, iou_loss = 0.000000, total_loss = 0.000048 
v3 (m